In [1]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV
from project_heart.enums import *

In [2]:
lv = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk")

In [3]:
lv.identify_regions(
  LV_GEO_TYPES.TYPE_B,
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)

lv.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value, 
        vnodes=[
          (LV_VIRTUAL_NODES.MITRAL, "green"),
          (LV_VIRTUAL_NODES.AORTIC, "red"),
          (LV_VIRTUAL_NODES.AORTIC_BORDER, "purple"),
          ],
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [ ]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC",
                                    LV_SURFS.BORDER_AORTIC,
                                    dist_from_c=5.0,
                                    r_alpha=0.7)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC",
                                    LV_SURFS.BORDER_MITRAL,
                                    dist_from_c=5.0,
                                    r_alpha=1.0)

aortic_epi_bc = lv.create_spring_rim_bc("AORTIC_BC2",
                                        LV_SURFS.EPI_AORTIC,
                                        dist_from_c=0.0,
                                        height=3.0,
                                        r_alpha=1.5)

In [ ]:
atr_rim_string_lines = lv.get_rim_springs_for_plot(aortic_bc)
mtr_rim_string_lines = lv.get_rim_springs_for_plot(mitral_bc)
aortic_epi_rim_string_lines = lv.get_rim_springs_for_plot(aortic_epi_bc)

In [ ]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node(LV_VIRTUAL_NODES.MITRAL), color="green", point_size=300)
plotter.add_points(lv.get_virtual_node(LV_VIRTUAL_NODES.AORTIC), color="red", point_size=300)

plotter.add_points(aortic_bc[LV_RIM.NODES.value], color="red", point_size=300)
plotter.add_points(aortic_bc[LV_RIM.CENTER.value], color="red", point_size=300)

plotter.add_points(mitral_bc[LV_RIM.NODES.value], color="green", point_size=300)
plotter.add_points(mitral_bc[LV_RIM.CENTER.value], color="green", point_size=300)

plotter.add_points(aortic_epi_bc[LV_RIM.NODES.value], color="pink", point_size=300)
plotter.add_points(aortic_epi_bc[LV_RIM.CENTER.value], color="pink", point_size=300)

plotter.add_mesh(atr_rim_string_lines, color="red", opacity=0.5, show_edges=False)
plotter.add_mesh(mtr_rim_string_lines, color="green", opacity=0.5, show_edges=False)
plotter.add_mesh(aortic_epi_rim_string_lines, color="pink", opacity=0.5, show_edges=False)

plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()